In [1]:
!pip install fashion-clip faiss-cpu

In [2]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to c:\users\smhrd\appdata\local\temp\pip-req-build-x0pk_hyz
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached ftfy-6.3.1-py3-none-any.whl.metadata (7.3 kB)
Using cached ftfy-6.3.1-py3-none-any.whl (44 kB)
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369571 sha256=278da36c18ef7be745e49a8b11fb5d0adbd3c390c30306594f1637edd982f44d
  Stored in directory: C:\Users\SMHRD\AppData\Local\Temp\pip-ephem-wheel-cache-1lak5h3u\wheels\35\3e\df\3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\SMHRD\AppData\Local\Temp\pip-req-build-x0pk_hyz'


In [3]:
import torch
import clip
from PIL import Image
import random
import matplotlib.pyplot as plt
import os

In [4]:
# ✅ 작업 디렉토리 설정
WorkDir = "back\projectmodel\project_model\image_classification"

<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\p'
C:\Users\SMHRD\AppData\Local\Temp\ipykernel_10672\2088214066.py:2: SyntaxWarning: invalid escape sequence '\p'
  WorkDir = "back\projectmodel\project_model\image_classification"


In [5]:
# ✅ 이미지 데이터 및 결과 저장 경로 설정
image_folder = os.path.join(WorkDir, "crawled_images")  # 원본 이미지 폴더
weather_result_folder = os.path.join(WorkDir, "weather_result")  # 체감 온도별 분류된 이미지 저장 폴더
training_data_folder = os.path.join(WorkDir, "training_data")  # 학습 데이터 저장 폴더

In [6]:
# 폴더 생성 (없으면 생성)
os.makedirs(weather_result_folder, exist_ok=True)
os.makedirs(training_data_folder, exist_ok=True)

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

c:\Users\SMHRD\anaconda3\Lib\site-packages\clip\clip.py:57: UserWarning: C:\Users\SMHRD/.cache/clip\ViT-B-32.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(f"{download_target} exists, but the SHA256 checksum does not match; re-downloading the file")
100%|███████████████████████████████████████| 338M/338M [00:04<00:00, 83.5MiB/s]


In [9]:
# ✅ 체감 온도 & 스타일 기반 텍스트 조건 생성
def generate_text_prompt(feels_like_temp, style):
    if feels_like_temp < 10:
        temp_desc = "cold weather outfit"
    elif 10 <= feels_like_temp <= 20:
        temp_desc = "mild weather outfit"
    else:
        temp_desc = "hot weather outfit"

    return f"{temp_desc} {style} style outfit"

In [10]:
# ✅ 폴더에서 의류 이미지 경로 불러오기
def load_image_paths(folder_path):
    image_dataset = {"Top": [], "Bottom": [], "Outerwear": []}
    for category in image_dataset.keys():
        category_path = os.path.join(folder_path, category)
        if os.path.exists(category_path):
            image_dataset[category] = [os.path.join(category_path, img) for img in os.listdir(category_path) if img.endswith((".jpg", ".png"))]
    return image_dataset

In [11]:
# ✅ Fashion-CLIP을 활용한 의류 이미지 분류
def classify_images_by_similarity(image_paths, prompt):
    images = [Image.open(img_path) for img_path in image_paths]
    image_tensors = torch.stack([preprocess(img) for img in images]).to(device)

    text = clip.tokenize([prompt]).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image_tensors)
        text_features = model.encode_text(text)
        similarities = (image_features @ text_features.T).squeeze(1)

    sorted_indices = similarities.argsort(descending=True)
    sorted_images = [image_paths[idx] for idx in sorted_indices]
    return sorted_images

In [12]:
# ✅ 체감 온도에 따라 이미지 저장
def save_weather_classified_images(feels_like_temp, style, image_folder, output_folder):
    image_dataset = load_image_paths(image_folder)
    prompt = generate_text_prompt(feels_like_temp, style)
    classified_outfit = {}

    for category, image_paths in image_dataset.items():
        if image_paths:
            sorted_images = classify_images_by_similarity(image_paths, prompt)
            classified_outfit[category] = sorted_images[:3]  # 상위 3개 이미지 선택

            category_output_folder = os.path.join(output_folder, f"{feels_like_temp}_{style}", category)
            os.makedirs(category_output_folder, exist_ok=True)

            for img_path in sorted_images[:3]:
                img = Image.open(img_path)
                img.save(os.path.join(category_output_folder, os.path.basename(img_path)))

In [14]:
# ✅ 학습 데이터 저장 (향후 모델 개선 시 사용 가능)
def save_training_data(image_folder, output_folder):
    image_dataset = load_image_paths(image_folder)
    for category, image_paths in image_dataset.items():
        category_folder = os.path.join(output_folder, category)
        os.makedirs(category_folder, exist_ok=True)
        for img_path in image_paths:
            img = Image.open(img_path)
            img.save(os.path.join(category_folder, os.path.basename(img_path)))

In [24]:
# ✅ Fashion-CLIP을 활용한 유사도 계산 함수
def classify_image_by_similarity(image_path, prompt):
    """주어진 이미지와 텍스트 프롬프트 간의 유사도를 계산"""
    image = Image.open(image_path)
    image_tensor = preprocess(image).unsqueeze(0).to(device)
    text = clip.tokenize([prompt]).to(device)
    
    with torch.no_grad():
        image_feature = model.encode_image(image_tensor)
        text_feature = model.encode_text(text)
        similarity = (image_feature @ text_feature.T).item()
    
    return similarity, image

In [25]:
# 학습 데이터 저장 실행
save_training_data(weather_result_folder, training_data_folder)

In [26]:
test_image_path = r"C:\\Users\\SMHRD\\Desktop\\5g\\5G\\back\\projectmodel\\project_model\\image_classification\\crawled_images\\image_4.jpg"

In [27]:
# ✅ 체감 온도 및 스타일 설정
feels_like_temp = 12  # 체감 온도
style = "Casual"  # 선택 스타일

In [ ]:
# ✅ 테스트 실행
prompt = generate_text_prompt(feels_like_temp, style)
similarity_score, test_image = classify_image_by_similarity(test_image_path, prompt)

: 

In [1]:
# ✅ 결과 출력
print(f"이미지 경로: {test_image_path}")
print(f"프롬프트: {prompt}")
print(f"유사도 점수: {similarity_score:.4f}")

# ✅ 이미지 출력
plt.figure(figsize=(5,5))
plt.imshow(test_image)
plt.axis("off")
plt.title(f"Similarity: {similarity_score:.4f}")
plt.show()


NameError: name 'test_image_path' is not defined